In [1]:
import pandas as pd
import numpy as np

# plots
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# processing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score

# modeling
from sklearn.linear_model import RidgeClassifier

# others
import warnings
warnings.filterwarnings('ignore')

# Homework 4

In [2]:
df = pd.read_csv('data/reddit_200k_train.csv', encoding = 'latin-1', index_col='Unnamed: 0')

# subset the columns
df['removed'] = df.REMOVED
df = df[['body', 'removed']]

In [3]:
df.head(2)

,body,removed
1,I've always been taught it emerged from the ea...,False
2,"As an ECE, my first feeling as ""HEY THAT'S NOT...",True


# Task 1: Bag of Words and Simple Features

### 1.1 Baseline Model

In [4]:
X = CountVectorizer().fit_transform(df.body)
y = np.where(df.removed, 1, 0)

In [6]:
baseline_score = cross_val_score(RidgeClassifier(solver='sag'), X, y, cv=5, scoring='roc_auc')

In [10]:
print('Baseline model achieves a mean of {} ROC-AUC on our training data.'.format(np.round(np.mean(baseline_score), 2)))

Baseline model achieves a mean of 0.73 ROC-AUC on our training data.


### 1.2 Processing